In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
brand_category_df = pd.read_csv('brand_category.csv')
categories_df = pd.read_csv('categories.csv')
offer_retailer_df = pd.read_csv('offer_retailer.csv')

# Merge datasets
merged_df = pd.merge(offer_retailer_df, brand_category_df, on='BRAND', how='left')
merged_df = pd.merge(merged_df, categories_df, on='PRODUCT_CATEGORY', how='left')

# Text preprocessing and feature extraction
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_df['OFFER'])

# Function to get relevant offers based on user input
def get_relevant_offers(user_input):
    user_tfidf = tfidf_vectorizer.transform([user_input])

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # Combine scores with offer details
    merged_df['similarity_score'] = similarity_scores

    # Sort by similarity score
    sorted_offers = merged_df.sort_values(by='similarity_score', ascending=False)

    return sorted_offers[['OFFER', 'RETAILER', 'BRAND', 'PRODUCT_CATEGORY', 'similarity_score']]

# Example usage
user_query = input("Enter your search: ")
relevant_offers = get_relevant_offers(user_query)

# Display the top relevant offers
print(relevant_offers.head())


Enter your search: Mature
                                                 OFFER              RETAILER  \
0       Spend $50 on a Full-Priced new Club Membership             SAMS CLUB   
554  Fresh Bakery Products at Casey's, select varie...  CASEYS GENERAL STORE   
544                   Brita® Standard OR Elite Filters                   NaN   
545                   Brita® Standard OR Elite Filters                   NaN   
546                                Spend $300 at Shaws                 SHAWS   

                    BRAND             PRODUCT_CATEGORY  similarity_score  
0               SAMS CLUB                          NaN               0.0  
554  CASEYS GENERAL STORE  Frozen Pizza & Pizza Snacks               0.0  
544                 BRITA                        Water               0.0  
545                 BRITA           Household Supplies               0.0  
546                 SHAWS             Cooking & Baking               0.0  
